In [2]:
import os
import json
import numpy as np
import torch
import result
import pandas as pd
from rnn_model import LSTMNet, GRUNet
from utils import DotDict, Logger, rmse, rmse_tensor, rmse_np, boolean_string, get_dir, get_time, next_dir, model_dir, get_model
from get_dataset import get_time_data, get_stnn_data
from stnn import *
import matplotlib.pyplot as plt
from matplotlib import axis
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
%matplotlib inline

%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# pred 3 time step

## Best Jar

In [3]:
best_jar_3_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'v4-output', 'jar_v4'))
jar_result = result.Printer(best_jar_3_folder)
pd.options.display.max_rows = None
jar_df = jar_result.get_df(col=['minrmse', 'minsum', 'min_rmse_epoch', 'activation', 'data_normalize', 'nlayers', 'nhid', 'activation', 'nt_train', 'validation_length', 'dataset'], increase=False).sort_values('minrmse')
jar_df

,minrmse,minsum,min_rmse_epoch,activation,data_normalize,nlayers,nhid,activation,nt_train,validation_length,dataset
STNN-I-stnn_09-13-20-31-03_9405,61.4557,1015.55,6671,sigmoid,d,1,0,sigmoid,8,3,jar
STNN-I-stnn_09-13-20-31-03_7200,82.408,1542.26,8413,sigmoid,d,1,0,sigmoid,8,3,jar
STNN-I-stnn_09-13-20-31-03_8999,86.464,1656.87,7774,sigmoid,d,1,0,sigmoid,8,3,jar
STNN-I-stnn_09-13-20-31-03_5817,139.293,2430.43,4976,sigmoid,d,1,0,sigmoid,12,3,jar
STNN-I-stnn_09-13-20-31-03_7800,143.854,2504.56,3866,sigmoid,d,1,0,sigmoid,12,3,jar
STNN-I-stnn_09-13-20-31-03_7197,144.341,2485.43,8603,sigmoid,d,1,0,sigmoid,12,3,jar
STNN-I-stnn_09-13-20-31-03_0226,166.066,756.417,9931,sigmoid,d,4,100,sigmoid,8,3,jar
STNN-I-stnn_09-13-20-31-03_9238,172.616,421.39,2521,tanh,d,3,100,tanh,8,3,jar
STNN-I-stnn_09-13-20-31-03_1123,177.522,219.745,3136,tanh,d,2,100,tanh,8,3,jar
STNN-I-stnn_09-13-20-31-03_7370,179.762,1268.55,9886,tanh,d,1,0,tanh,8,3,jar


In [7]:
result.output_scr_by_dir({"v4": "STNN-I-stnn_09-13-20-31-03_7800"}, "../v4-output/jar_v4", minepoch="rmse", write="a")

In [5]:
best_jar_7_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'v4-output', 'jar_v4'))
jar_result = result.Printer(best_jar_7_folder)
pd.options.display.max_rows = None
jar_df = jar_result.get_df(col=['minrmse', 'minsum', 'min_rmse_epoch', 'activation', 'data_normalize', 'nt_train', 'validation_length', 'dataset', 'nx'], increase=False).sort_values('minrmse')
jar_df

,minrmse,minsum,min_rmse_epoch,activation,data_normalize,nt_train,validation_length,dataset,nx
STNN-I-stnn_09-13-20-31-03_9405,61.4557,1015.55,6671,sigmoid,d,8,3,jar,31
STNN-I-stnn_09-13-20-31-03_7200,82.408,1542.26,8413,sigmoid,d,8,3,jar,31
STNN-I-stnn_09-13-20-31-03_8999,86.464,1656.87,7774,sigmoid,d,8,3,jar,31
STNN-I-stnn_09-13-20-31-03_5817,139.293,2430.43,4976,sigmoid,d,12,3,jar,31
STNN-I-stnn_09-13-20-31-03_7800,143.854,2504.56,3866,sigmoid,d,12,3,jar,31
STNN-I-stnn_09-13-20-31-03_7197,144.341,2485.43,8603,sigmoid,d,12,3,jar,31
STNN-I-stnn_09-13-20-31-03_0226,166.066,756.417,9931,sigmoid,d,8,3,jar,31
STNN-I-stnn_09-13-20-31-03_9238,172.616,421.39,2521,tanh,d,8,3,jar,31
STNN-I-stnn_09-13-20-31-03_1123,177.522,219.745,3136,tanh,d,8,3,jar,31
STNN-I-stnn_09-13-20-31-03_7370,179.762,1268.55,9886,tanh,d,8,3,jar,31


# Feb

In [10]:
best_feb_3_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'v4-output', 'feb_v4'))
feb_result = result.Printer(best_feb_3_folder)
pd.options.display.max_rows = None
feb_df = feb_result.get_df(col=['minrmse', 'minsum', 'min_rmse_epoch', 'activation', 'data_normalize', 'nlayers', 'nhid', 'activation', 'nx'], increase=False).sort_values('minrmse')
feb_df

,minrmse,minsum,min_rmse_epoch,activation,data_normalize,nlayers,nhid,activation,nx
STNN-I-stnn_09-11-18-29-27_6337,17.1661,136.109,7377,sigmoid,d,1,0,sigmoid,31
STNN-I-stnn_09-11-18-29-27_7763,23.3962,32.1772,8539,sigmoid,d,1,0,sigmoid,31
STNN-I-stnn_09-11-18-29-27_0455,28.6264,129.823,5854,sigmoid,d,1,0,sigmoid,31
STNN-I-stnn_09-11-18-29-27_3759,41.2153,122.123,9877,sigmoid,d,2,100,sigmoid,31
STNN-I-stnn_09-11-18-29-27_9812,47.0322,94.7019,7143,sigmoid,d,2,100,sigmoid,31
STNN-I-stnn_09-11-18-29-27_3887,47.5041,185.324,4845,tanh,d,1,0,tanh,31
STNN-I-stnn_09-11-18-29-27_4103,49.1125,217.502,4784,sigmoid,d,2,100,sigmoid,31
STNN-I-stnn_09-11-18-29-27_8322,50.741,322.377,2976,sigmoid,d,4,100,sigmoid,31
STNN-I-stnn_09-11-18-29-27_0644,52.8824,427.667,1451,sigmoid,d,2,100,sigmoid,31
STNN-I-stnn_09-11-18-29-27_0616,53.119,521.69,9689,sigmoid,d,1,0,sigmoid,31


In [9]:
result.output_scr_by_dir({"v4": "STNN-I-stnn_09-11-18-29-27_3759"}, "../v4-output/feb_v4", minepoch="rmse", write='a')

In [11]:
best_feb_7_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'v4-output', 'feb_v4_7'))
feb_result = result.Printer(best_feb_7_folder)
pd.options.display.max_rows = None
feb_df = feb_result.get_df(col=['minrmse', 'minsum', 'min_rmse_epoch', 'activation', 'data_normalize', 'nlayers', 'nhid', 'activation', 'nx'], increase=False).sort_values('minrmse')
feb_df

,minrmse,minsum,min_rmse_epoch,activation,data_normalize,nlayers,nhid,activation,nx
STNN-I-stnn_09-11-18-31-28_3471,94.5472,426.513,647,tanh,d,3,100,tanh,31
STNN-I-stnn_09-11-18-31-28_1136,99.6479,172.714,782,tanh,d,3,100,tanh,31
STNN-I-stnn_09-11-18-31-28_5883,107.584,293.457,775,tanh,d,3,100,tanh,31
STNN-I-stnn_09-11-18-31-28_2412,136.435,163.279,872,tanh,d,2,100,tanh,31
STNN-I-stnn_09-11-18-31-28_0883,151.973,486.209,7575,sigmoid,d,3,100,sigmoid,31
STNN-I-stnn_09-11-18-31-28_0985,156.424,357.676,1817,tanh,d,4,100,tanh,31
STNN-I-stnn_09-11-18-31-28_0351,161.554,700.164,2314,sigmoid,d,4,100,sigmoid,31
STNN-I-stnn_09-11-18-31-28_7279,162.872,616.447,844,tanh,d,2,100,tanh,31
STNN-I-stnn_09-11-18-31-28_0686,165.012,1026.29,1911,sigmoid,d,3,100,sigmoid,31
STNN-I-stnn_09-11-18-31-28_5278,168.107,726.507,817,sigmoid,d,2,100,sigmoid,31


In [10]:
result.output_scr_by_dir({"v4": "STNN-I-stnn_09-11-18-31-28_3471"}, "../v4-output/feb_v4_7", minepoch="rmse", write='a')

In [14]:
best_mar_7_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'v4-output', 'mar_v4'))
mar_result = result.Printer(best_mar_7_folder)
pd.options.display.max_rows = None
mar_df = mar_result.get_df(col=['minrmse', 'minsum', 'min_rmse_epoch', 'activation', 'data_normalize', 'nlayers', 'nhid', 'activation', 'nx', 'validation_length'], increase=False).sort_values('minrmse')
mar_df

,minrmse,minsum,min_rmse_epoch,activation,data_normalize,nlayers,nhid,activation,nx,validation_length
STNN-I-stnn_09-11-17-23-42_6930,13.1358,4.57146,3924,tanh,d,2,100,tanh,31,3
STNN-I-stnn_09-11-18-26-17_4928,20.5967,18.6598,3510,tanh,d,2,100,tanh,31,7
STNN-I-stnn_09-11-17-23-42_5704,23.3093,5.93123,1563,tanh,d,2,100,tanh,31,3
STNN-I-stnn_09-11-17-23-42_5250,24.738,8.82637,2486,tanh,d,3,100,tanh,31,3
STNN-I-stnn_09-11-18-26-17_9282,26.0585,9.64486,2590,tanh,d,2,100,tanh,31,7
STNN-I-stnn_09-11-18-26-17_1799,33.3547,17.9953,8998,tanh,d,2,100,tanh,31,7
STNN-I-stnn_09-11-18-26-17_4304,34.0297,51.8503,9963,sigmoid,d,1,0,sigmoid,31,7
STNN-I-stnn_09-11-18-26-17_2662,34.2379,14.0326,4243,tanh,d,3,100,tanh,31,7
STNN-I-stnn_09-11-17-23-42_0165,35.3643,12.6799,8660,tanh,d,2,100,tanh,31,3
STNN-I-stnn_09-11-18-26-17_6177,35.9578,60.3285,9711,sigmoid,d,1,0,sigmoid,31,7


In [15]:
result.output_scr_by_dir({"v4": "STNN-I-stnn_09-11-17-23-42_6930"}, "../v4-output/mar_v4", minepoch="rmse", write='a')